# Install Prerequisites

This notebook installs packages needed by subsequent notebooks. On your SageMaker notebook instance, you must:

1. open this notebook with the `conda_mxnet_p36` kernel,
2. then execute the cells 

**NOTE**: you must always rerun this notebook after you restart your notebook instance.

In [ ]:
%pip uninstall --yes enum34
%pip install gluonts "git+https://github.com/aws-samples/smallmatter-package@master#egg=smallmatter"

Download data

In [ ]:
from gluonts.dataset.repository import datasets
from pathlib import Path
dataset = datasets.get_dataset('m4_weekly', path=Path('../data/'))

Next steps:
    
1. Convert to .csv
2. Bump time from start 1750-01-01 00:00:00 to something saner
3. Remember this is weekly, so align to W-SUN
4. Simulate holes.